In [ ]:
import cv2
from matplotlib import pyplot as plt

Opening an Image


In [ ]:

image_file = 'op7.png'
img = cv2.imread(image_file)
# cv2.imshow("original image", img)

displaying diff images with actual size in matplotlib subplot

In [ ]:

def display(im_path):
    dpi=80
    im_data = plt.imread(im_path)
    height, width, depth = im_data.shape

    figsize = width / float(dpi), height/float(dpi)

    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0,0,1,1])
    
    #hide spins, ticks, etc
    ax.axis('off')

    ax.imshow(im_data, cmap = 'gray')
    plt.show()


def display1(im_path):
    dpi=80
    im_data = plt.imread(im_path)
    height, width = im_data.shape

    figsize = width / float(dpi), height/float(dpi)

    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0,0,1,1])
    
    #hide spins, ticks, etc
    ax.axis('off')

    ax.imshow(im_data, cmap = 'gray')
    plt.show()

In [ ]:
#call the function to show image
display(image_file)

INVERTED IMAGE

In [ ]:
inverted_image = cv2.bitwise_not(img)
cv2.imwrite('inverted.png', inverted_image)

In [ ]:
display('inverted.png') 

RESCALING IMAGE

In [ ]:
pass

BINARIZATION

In [ ]:
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
gray_image = grayscale(img)
cv2.imwrite('gray.png', gray_image)

In [ ]:
display1('gray.png')

In [ ]:
thresh ,im_bw = cv2.threshold(gray_image, 200, 230, cv2.THRESH_BINARY)
cv2.imwrite('b-w.jpg', im_bw)

NOISE REMOVAL

In [ ]:
def noise_removal(image):
    import numpy as np
    kernal = np.ones((1,1), np.uint8)
    image = cv2.dilate(image, kernal,  iterations=1)
    kernal = np.ones((1,1), np.uint8)
    image = cv2.erode(image, kernal ,iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernal)
    image = cv2.medianBlur(image, 3)
    return (image)

In [ ]:
no_noise = noise_removal(im_bw)
cv2.imwrite('no_noise.jpg', no_noise)

In [ ]:
display1('no_noise.jpg')

DILATION and EROSION

In [ ]:
def thin_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernal = np.ones((2, 2), np.uint8)
    image = cv2.erode(image, kernal , iterations=1)
    image = cv2.bitwise_not(image)
    return (image)

In [ ]:
eroded_image = thin_font(no_noise)
cv2.imwrite('eroded_img.jpg', eroded_image)

In [ ]:
display1('eroded_img.jpg')

In [ ]:
def thick_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernal = np.ones((2, 2), np.uint8)
    image = cv2.dilate(image, kernal , iterations=1)
    image = cv2.bitwise_not(image)
    return (image)

In [ ]:
dilated_image = thick_font(no_noise)
cv2.imwrite('dilated.jpg', dilated_image)

In [ ]:
display1('dilated.jpg')

ROTATION and DESKEWING

In [64]:
pass

REMOVING BORDERS

In [65]:
def remove_borders(image):
    contours, heiarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contSorted = sorted(contours, key = lambda x:cv2.contourArea(x))
    cnt = contSorted[-1]
    x, y, w, h = cv2.boundingRect(cnt)
    crop = image[y:y+h, x:x+w]
    return (crop)

In [66]:
no_borders = remove_borders(no_noise)
cv2.imwrite('no-borders.jpg', no_borders)

True

In [ ]:
display1('no-borders.jpg')

MISSING BORDERS

In [70]:
color = [255,255,255]
top , bottom, left, right   = [150]*4

In [72]:
image_with_border = cv2.copyMakeBorder(no_borders, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
cv2.imwrite('image_with_border.jpg', image_with_border)

True

In [ ]:
display1('image_with_border.jpg')

In [75]:
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot as plt
from pytesseract import Output
from tabulate import tabulate
import pandas as pd
import numpy as np
import pytesseract
import argparse
import imutils
import cv2

In [ ]:
# set the PSM mode to detect sparse text, and then localize text in
# the table
img = ''
options = "--psm 6"
results = pytesseract.image_to_data(
	cv2.cvtColor(table, cv2.COLOR_BGR2RGB),
	config=options,
	output_type=Output.DICT)

# initialize a list to store the (x, y)-coordinates of the detected
# text along with the OCR'd text itself
coords = []
ocrText = []

# loop over each of the individual text localizations
for i in range(0, len(results["text"])):
	# extract the bounding box coordinates of the text region from
	# the current result
	x = results["left"][i]
	y = results["top"][i]
	w = results["width"][i]
	h = results["height"][i]

	# extract the OCR text itself along with the confidence of the
	# text localization
	text = results["text"][i]
	conf = int(float(results["conf"][i]))

	# filter out weak confidence text localizations
	if conf > args["min_conf"]:
		# update our text bounding box coordinates and OCR'd text,
		# respectively
		coords.append((x, y, w, h))
		ocrText.append(text)